In [22]:
1+1

2

In [29]:
from langgraph.graph import StateGraph,END,START
from typing import TypedDict,Annotated
from langchain_core.messages import BaseMessage,HumanMessage
from langchain_ollama import ChatOllama
from langgraph.graph.message import BaseMessage,add_messages
from langgraph.checkpoint.memory import MemorySaver

In [30]:
class Chatstate(TypedDict):
    messages: Annotated[list[BaseMessage],add_messages]

In [31]:
llm=ChatOllama(model='llama3.2:1b',temperature=0.2,num_predict=256)

def chat_node(state:Chatstate):

    messages=state['messages']
    response=llm.invoke(messages)

    return {'messages':[response]}

In [32]:
checkpointer=MemorySaver()

graph=StateGraph(Chatstate)

graph.add_node('chat_node',chat_node)

graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

chatbot=graph.compile(checkpointer=checkpointer)

In [27]:
initail_state={'messages':HumanMessage(content="what is the capital of india")}

In [ ]:
chatbot.invoke(initail_state)['messages'][-1].content

In [34]:
thread_id='1'
while True:
    user_message=input("type here :")
    print("User :",user_message)
    if user_message.strip().lower() in ['exit','quite','byy']:
        break
    
    config={'configurable':{'thread_id':thread_id}}

    responce=chatbot.invoke({'messages':[HumanMessage(content=user_message)]},config=config)

    print("AI :",responce['messages'][-1].content)

User : hi
AI : How can I help you today?
User : my name is abdullah
AI : Hello Abdullah. How are you doing today? Is there something I can help you with or would you like to chat?
User : what ismy name
AI : Your name is Abdullah.
User : can you add 100 to 10
AI : To add 100 to 10, I'll do the calculation:

10 + 100 = 110
User : now you multiply result with 2
AI : I'll multiply 110 by 2:

110 × 2 = 220
User : exit


In [35]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='a938e2c2-9572-4d42-877b-0b79bca23859'), AIMessage(content='How can I help you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-27T14:08:28.9594783Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7786213100, 'load_duration': 6386961700, 'prompt_eval_count': 26, 'prompt_eval_duration': 803738500, 'eval_count': 8, 'eval_duration': 551053000, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--404af751-75e1-4086-a4e5-7489666cb290-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34}), HumanMessage(content='my name is abdullah', additional_kwargs={}, response_metadata={}, id='71c0d54e-31d5-43fb-90d8-07ec38c657f1'), AIMessage(content='Hello Abdullah. How are you doing today? Is there something I can help you with or would you like to chat?', additional_kwargs={}, re